In [1]:
!git clone https://github.com/JaidedAI/EasyOCR.git && cd EasyOCR

Cloning into 'EasyOCR'...
remote: Enumerating objects: 2750, done.
remote: Counting objects: 100% (700/700), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 2750 (delta 646), reused 632 (delta 632), pack-reused 2050 (from 1)
Receiving objects: 100% (2750/2750), 157.83 MiB | 10.90 MiB/s, done.
Resolving deltas: 100% (1686/1686), done.
Updating files: 100% (313/313), done.


In [1]:
import os
os.chdir('/content/EasyOCR/trainer')
%cd /content/EasyOCR/trainer
!pip install torch==2.2.2 pandas torchvision==0.17.2
!mv /content/EasyOCR/trainer/test.py /content/EasyOCR/trainer/test2.py
!sed -i 's/from test import validation/from test2 import validation/g' /content/EasyOCR/trainer/train.py
!sed -i 's/image, text = data_loader_iter.next()/image, text = next(data_loader_iter)/g' /content/EasyOCR/trainer/dataset.py
!sed -i 's/image, text = self.dataloader_iter_list[i].next()/image, text = next(self.dataloader_iter_list[i])/g' /content/EasyOCR/trainer/dataset.py
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

/content
mv: cannot stat '/content/EasyOCR/trainer/test.py': No such file or directory


In [2]:
cudnn.benchmark = True
cudnn.deterministic = False

In [ ]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [ ]:
opt = get_config("config_files/en_filtered_config.yaml")
train(opt, amp=False)